In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matminer.datasets import load_dataset
from matminer.featurizers.composition import ElementProperty
from pymatgen.core.composition import Composition
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
df = load_dataset("elastic_tensor_2015")
ep = ElementProperty.from_preset("magpie")
df["composition"] = df["formula"].apply(Composition)
df = ep.featurize_dataframe(df, "composition")
df = df.dropna()


In [ ]:
# Use "K_VRH" as the target property
target = "K_VRH"

# Drop all non-numeric columns
X = df.select_dtypes(include='number').drop(columns=[target])
y = df[target]

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
preds = model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, preds)
rmse = mse ** 0.5
r2 = r2_score(y_test, preds)

print("RMSE:", rmse)
print("R² Score:", r2)


In [ ]:
plt.scatter(y_test, preds)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Bulk Modulus Prediction")
plt.grid(True)
plt.show()


In [ ]:
importances = model.feature_importances_
feature_names = X.columns
top_features = sorted(zip(importances, feature_names), reverse=True)[:5]
for score, name in top_features:
    print(f"{name}: {score:.4f}")


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge

models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Ridge Regression": Ridge()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds) ** 0.5
    r2 = r2_score(y_test, preds)
    results[name] = {"RMSE": rmse, "R²": r2}

pd.DataFrame(results).T


In [ ]:
%pip install seaborn

import seaborn as sns

# Use the Random Forest model from above
rf_model = models["Random Forest"]
feat_imp = pd.Series(rf_model.feature_importances_, index=X.columns)
top_feat = feat_imp.sort_values(ascending=False).head(10)

sns.barplot(x=top_feat.values, y=top_feat.index)
plt.title("Top 10 Important Features")
plt.xlabel("Importance")
plt.show()


In [ ]:
plt.scatter(y_test, rf_model.predict(X_test), alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.xlabel("Actual Bulk Modulus")
plt.ylabel("Predicted Bulk Modulus")
plt.title("Random Forest: Actual vs Predicted")
plt.grid(True)
plt.show()
